# Halloween Strategy
[Use This 'Scary Indicator' To Trounce The U.S. Stock Market](https://seekingalpha.com/article/4118841-use-scary-indicator-trounce-u-s-stock-market)

In [ ]:
library(xts)
library(zoo)
library(TTR)
library(dplyr)
library(quantmod)
setwd('/Users/jtan/Documents/Investment/Halloween-Indicator')

output_table <- data.frame (
    Date=character(),
    Symbol=character(),
    Return=numeric(),
    Period=numeric(),
    stringsAsFactors=FALSE
)

In [3]:
STOCK_LIST <- c('EEM','VWO','VEA','SPY','QQQ')
START_DATE <- '2000-05-01' # Start from at least one year ago from end_date
END_DATE <- '2017-05-01' # End on April (Nov 1 to May 1)

# table initialization
result_table <- output_table

getSymbols(STOCK_LIST, from=START_DATE, to=END_DATE)

for(symbol in STOCK_LIST) {
    
    cat('Processing', symbol,'\n')
    
    x <- get(symbol)
    x <- to.monthly(x, indexAt='lastof', drop.time=TRUE) 
    indexFormat(x) <- '%Y-%m-%d'
    colnames(x) <- gsub("x", symbol, colnames(x))
    x <- x[,4]  # Drops all columns except close price
    assign(symbol, x)
    
    test <- data.frame(Date = index(x), x, row.names=NULL)
    test$mo <- as.numeric(format(test$Date, "%m"))
    colnames(test)[2] <- 'Close'
    test$Symbol <- symbol
    
    new_05to10 <- test %>% filter(mo==5|mo==10) %>% arrange(Date) %>% 
                    mutate(Return = (Close - lag(Close, default = Close[1])) / lag(Close, default = Close[1])) %>% 
                    filter(mo==10) #%>% select(-c(mo,Close)) 
    new_05to10$Period <- as.numeric(format(new_05to10$Date, "%Y"))
    new_05to10 <- new_05to10[c('Date','Symbol','Return','Period')]
    
    new_11to04 <- test %>% filter(mo==4|mo==11) %>% arrange(Date) %>% 
                    mutate(Return = (Close - lag(Close, default = Close[1])) / lag(Close, default = Close[1])) %>% 
                    filter(mo==4) #%>% select(-c(mo,Close))     
    new_11to04$Period <- as.numeric(format(new_11to04$Date, "%Y")) - 1
    new_11to04 <- new_11to04[c('Date','Symbol','Return','Period')]
    
    y <- merge(new_05to10,new_11to04,by=c('Symbol','Period'))
    result_table <- rbind(y, result_table)
}


[1] "EEM" "EET"

[1] "EEM"
[1] "EET"


In [ ]:
write.table(result_table, file = 'Indicator_Comparison.csv', sep = ",", col.names = TRUE, row.names = FALSE, quote = FALSE)